In [22]:
#encoding=utf-8
import pandas as pd
import math

loans = pd.read_csv('loans.csv')
del loans['id']

In [85]:
loans

,age,job,house,credit,class
0,青年,否,否,一般,否
1,青年,否,否,好,否
2,青年,是,否,好,是
3,青年,是,是,一般,是
4,青年,否,否,一般,否
5,中年,否,否,一般,否
6,中年,否,否,好,否
7,中年,是,是,好,是
8,中年,否,是,非常好,是
9,中年,否,是,非常好,是


In [3]:
def change(y):
    x = y.decode('utf-8')
    if x == u'青年':
        x = 0
    elif x == u'中年':
        x = 1
    elif x == u'老年':
        x = 2
    elif x == u'否':
        x = 0
    elif x == u'是':
        x = 1
    elif x == u'一般':
        x = 0
    elif x == u'好':
        x = 1
    elif x == u'非常好':
        x = 2
    return x

n_loans = loans.applymap(change)

In [129]:
def gen_P_x(df=None):
    """ 计算每个特征的概率 P（X=xi）,并返回一个dict """
    if df is None:
        df = pd.DataFrame()
    P_x = dict()
    raw_num = df.shape[0]
    for X in df.columns:
        P_x[X] = loans[X].value_counts().apply(lambda x: float(x)/raw_num)
    return P_x
 
def gen_H_x(p_x=None):
    """ 计算经验熵 """
    if p_x is None:
        p_x = dict()
    H_x = dict()
    for key in p_x:
        ss = p_x[key].apply(lambda x: x*math.log(x, 2) if x>0.0000000001 else 0)
        H_x[key]= -ss.agg('sum')
    return H_x

def gen_P_x_y(df=None, Y=None):
    """ 计算经验条件概率分布 P(X|Y) = P(XY)/P(X)"""
    if df is None or Y is None:
        return None
    if Y not in df.columns:
        return None
    # 需要对不同的特征X结合Y（分类）的取值进行计算条件概率
    x_list = [ item for item in df.columns if item != Y]
    raw_num = df.shape[0]
    p_x = gen_P_x(df)
    p_y = p_x[Y]
    del p_x[Y]  # 删除Y所在的列
    P_x_y = dict()
    for x in x_list:
        sub_df = df[[x, Y]]
        p_x_y = dict()
        for y in p_y.index:
            ss = sub_df[sub_df[Y]==y][x].value_counts().apply(lambda x: float(x)/raw_num)
            p_x_y[y.decode('utf-8')] = p_x[x].combine(other=ss, func=lambda x,y: y/x)
        P_x_y[x] = p_x_y
    return P_x_y

def gen_H_x_y(df=None, X=None, Y=None):
    """计算条件经验熵，含义是指已知X特征的条件下，随机变量Y的不确定性的大小。
    """
    p_x_y = gen_P_x_y(df, Y)
    res = None
    for key in p_x_y[X]:
        ss =  p_x_y[X][key]
        if res is not None:
            res += ss.apply(lambda x: x*math.log(x, 2) if x>0.0000000001 else 0)
        else:
            res = ss.apply(lambda x: x*math.log(x, 2) if x>0.0000000001 else 0)
    return res.apply(abs)

def gen_info_gain(df=None, X=None, Y=None):
    """ 计算选择特征X判断时的信息增益 g(D,X) = H(X) - H(D|X)"""
    h_x_y = gen_H_x_y(df, X, Y)
  
    p_y_all = gen_P_x(df)
    p_X = p_y_all[X]
    h_Y = gen_H_x(p_y_all)[Y]
#     print "经验熵h_Y"
#     print h_Y
#     print '-'*20
#     print "条件经验熵h_x_y"
#     print h_x_y
#     print '-'*20
    ss = p_X.combine(other=h_x_y,func= lambda x,y: x*y).agg('sum')
    return h_Y - ss

# p_x = gen_P_x(loans)
# gen_H_x(p_x)
# p_x_y = gen_P_x_y(loans, u'class')
## 使用p_x_y计算info_gain

print "age info gain:%0.4f" % gen_info_gain(loans, 'age', 'class')
print "house info gain:%0.4f" % gen_info_gain(loans, 'house', 'class')
print "credit info gain:%0.4f" % gen_info_gain(loans, 'credit', 'class')
print "job info gain:%0.4f" % gen_info_gain(loans, 'job', 'class')



age info gain:0.0830
house info gain:0.4200
credit info gain:0.3630
job info gain:0.3237


中年    5
老年    5
青年    5
Name: age, dtype: int64

In [24]:
total_count  = len(loans);total_count




15